In [1]:
import os
import requests
from datetime import datetime, timedelta
from dotenv import load_dotenv


In [2]:
def search_events(artist_name, ticketmaster_api_key, ticketmaster_api_url, size=10):
    ticketmaster_params = {
        'apikey': ticketmaster_api_key,
        'keyword': artist_name,
        'size': size
    }
    
    ticketmaster_response = requests.get(ticketmaster_api_url, params=ticketmaster_params)

    upcoming_events = []

    if ticketmaster_response.status_code == 200:
        ticketmaster_data = ticketmaster_response.json()
        for event in ticketmaster_data.get('_embedded', {}).get('events', []):
            upcoming_events.append(event)
    else:
        print(f'Error: {ticketmaster_response.status_code}')

    if len(upcoming_events) == 0:
        return None

    return upcoming_events


def parse_event(band_name, event):
    band_name = band_name.lower()
    event_name = event['name']

    if 'dates' in event:
        if 'start' in event['dates'] and 'localDate' in event['dates']['start']:
            local_date = event['dates']['start']['localDate']
        else:
            local_date = "Unknown"

        if 'start' in event['dates'] and 'localTime' in event['dates']['start']:
            local_time = event['dates']['start']['localTime']
        else:
            local_time = "Unknown"

        if 'timezone' in event['dates']:
            timezone = event['dates']['timezone']
        else:
            timezone = "Unknown"

    else:
        local_date = "Unknown"
        local_time = "Unknown"
        timezone = "Unknown"

    if local_date == "Unknown" or local_time == "Unknown":
        event_datetime = None
        formatted_datetime = "Date and time unknown"
    else:
        event_datetime = datetime.strptime(f"{local_date} {local_time}", '%Y-%m-%d %H:%M:%S')
        formatted_datetime = event_datetime.strftime('%A, %B %d, %Y at %I:%M %p')\

    if '_embedded' in event and 'venues' in event['_embedded']:
        if event['_embedded']['venues']:
            venue_name = event['_embedded']['venues'][0].get('name', "Venue name not available")
            venue_address = event['_embedded']['venues'][0].get('address', {}).get('line1', "Venue address not available")
        else:
            venue_name = "Venue information not available"
            venue_address = "Venue information not available"

    else:
        venue_name = "Venue information not available"
        venue_address = "Venue information not available"

    event_info = {
        "band_name": band_name,
        "event_name": event_name,
        "date_and_time": f"{local_date} {local_time} ({timezone})",
        "start_datetime": formatted_datetime,
        "event_url": event.get('url', "URL not available"),
        "timezone": timezone,
        "locale": event.get('locale', "Locale not available"),
        "venue_name": venue_name,
        "venue_address": venue_address
    }

    return event_info

In [3]:
load_dotenv()

TICKETMASTER_API_KEY = os.getenv('TICKETMASTER_API_KEY')
TICKETMASTER_API_URL = 'https://app.ticketmaster.com/discovery/v2/events.json'

In [4]:
artists = ['Lorna Shore', 'The devil wears prada', 'gojira', 'thy art is murder', 'igorrr', 'bring me the horizon']

events_to_get = 3
for i in artists:
    events = search_events(i, TICKETMASTER_API_KEY, TICKETMASTER_API_URL, events_to_get)

    if events:
        for event in events:
            print(parse_event(band_name=i, event=event))
            print('_' * 200)

# events = search_events('Lorna Shore', TICKETMASTER_API_KEY, TICKETMASTER_API_URL, 3)

{'band_name': 'lorna shore', 'event_name': 'Lorna Shore', 'date_and_time': '2023-11-28 18:00:00 (Europe/London)', 'start_datetime': 'Tuesday, November 28, 2023 at 06:00 PM', 'event_url': 'https://www.ticketmaster.co.uk/lorna-shore-birmingham-28-11-2023/event/3E005ED3DCFE1758', 'timezone': 'Europe/London', 'locale': 'en-us', 'venue_name': 'O2 Academy Birmingham', 'venue_address': '16-18 Horsefair'}
________________________________________________________________________________________________________________________________________________________________________________________________________
{'band_name': 'lorna shore', 'event_name': 'Lorna Shore', 'date_and_time': '2023-11-29 18:00:00 (Europe/London)', 'start_datetime': 'Wednesday, November 29, 2023 at 06:00 PM', 'event_url': 'https://www.ticketmaster.co.uk/lorna-shore-leeds-29-11-2023/event/3E005ED4B65B1806', 'timezone': 'Europe/London', 'locale': 'en-us', 'venue_name': 'O2 Academy Leeds', 'venue_address': '55 Cookridge Street'}
_

In [5]:
events[0]        

{'name': 'Bring Me The Horizon',
 'type': 'event',
 'id': 'G5dbZ9ikrmspR',
 'test': False,
 'url': 'https://www.ticketmaster.co.uk/bring-me-the-horizon-cardiff-09-01-2024/event/36005EC19EBB15AF',
 'locale': 'en-us',
 'images': [{'ratio': '16_9',
   'url': 'https://s1.ticketm.net/dam/a/f1f/45a5ba30-fdb2-4a75-8f58-f88eec0bdf1f_EVENT_DETAIL_PAGE_16_9.jpg',
   'width': 205,
   'height': 115,
   'fallback': False},
  {'ratio': '4_3',
   'url': 'https://s1.ticketm.net/dam/a/f1f/45a5ba30-fdb2-4a75-8f58-f88eec0bdf1f_CUSTOM.jpg',
   'width': 305,
   'height': 225,
   'fallback': False},
  {'ratio': '4_3',
   'url': 'https://s1.ticketm.net/dam/a/dde/6e18d841-e654-4324-a173-db0e61d3adde_SOURCE',
   'width': 2426,
   'height': 1800,
   'fallback': False},
  {'ratio': '3_2',
   'url': 'https://s1.ticketm.net/dam/a/f1f/45a5ba30-fdb2-4a75-8f58-f88eec0bdf1f_TABLET_LANDSCAPE_3_2.jpg',
   'width': 1024,
   'height': 683,
   'fallback': False},
  {'ratio': '16_9',
   'url': 'https://s1.ticketm.net/dam/a/